# 🙀 Cat Evaluation

### 📝 Imports

In [8]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import albumentations as A

import torchvision.transforms as transforms

import os

from cat_discriminator_neural_net import CatDiscriminatorNeuralNet

from src.augmentation.data_augmenter import DataAugmenter

from cats_dataset import CatsDataset

### 🔧 Config

In [9]:
image_size = 512

saved_model_path = "trained_networks/cat_discriminator.pth"

testing_data_path = 'data/test'

### 🌐 Create Transforms

In [ ]:
transform = transforms.Compose([
    DataAugmenter(target_image_size=image_size, augment_images=False),
    transforms.ToTensor(), # converts numpy to trans
])

### 🤓📝 Load Testing Data

In [11]:
dataset = CatsDataset(
    root_dir=testing_data_path,
    transform=transform)

dataloader = DataLoader(dataset, shuffle=False, num_workers=4)

### 🥾 Initialize the Neural Net

In [ ]:
net = CatDiscriminatorNeuralNet()

if os.path.isfile(saved_model_path):
    net.load_state_dict(torch.load(saved_model_path))

net.cuda()
net.eval()

### 🤖 Evaluate

In [ ]:
import numpy


correct = 0
total = 0

def print_predictions(labels, outputs):
    numpy_labels = labels.flatten().cpu().numpy()
    max_label_index = numpy.argmax(numpy_labels)
    actual_label = dataset.index_to_class_name(max_label_index)

    numpy_outputs = outputs.flatten().cpu().numpy()
    max_output_index = numpy.argmax(numpy_outputs)
    predicted_label = dataset.index_to_class_name(max_output_index)


    print("Actual:" + actual_label)
    print("Predicted:" + predicted_label)

    for i in range(len(numpy_outputs)):
        print(f'{dataset.index_to_class_name(i)}: {numpy_outputs[i]:.2f}')

    print("")
    print("")

with torch.no_grad():
    for inputs, labels in dataloader:
        inputs = inputs.to('cuda')
        labels = labels.to('cuda')

        # Forward pass
        outputs = net(inputs)  # outputs: [batch_size, num_classes]

        # Get predicted classes as indices
        _, predicted = torch.max(outputs, dim=1)  

        print_predictions(labels, outputs)
        
        numpy_labels = labels.flatten().cpu().numpy()
        max_label_index = numpy.argmax(numpy_labels)
        actual_label = dataset.index_to_class_name(max_label_index)

        numpy_outputs = outputs.flatten().cpu().numpy()
        max_output_index = numpy.argmax(numpy_outputs)
        predicted_label = dataset.index_to_class_name(max_output_index)

        total += 1
        if predicted_label == actual_label:
            correct += 1

accuracy = (correct / total) * 100
print(f'Accuracy: {accuracy:.2f}%')

### 💾 Save Progress

In [14]:
torch.save(net.state_dict(), saved_model_path)